In [1]:
# Required packages
import pandas as pd
import numpy as np
import geopandas as gpd

# Copy
import copy 

# Imputing Data
from sklearn.impute import SimpleImputer

# Progress Bar
from IPython.core.display import Image, display
import progressbar

# Plots
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['text.color'] = 'k'
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# Calgary Property Assessments


The assessed values of residential, non-residential and farm land properties in Calgary. The properties in this dataset consist of Calgary lands that have a registered parcel at Alberta’s Land Titles Office. Properties that are on The City of Calgary’s annual property assessment rolls, but excluded from this dataset, are titled parking stalls, titled storage units, machinery & equipment property, and linear property (as defined in the Municipal Government Act). This dataset can be downloaded from [here](https://data.calgary.ca/dataset/Property-Assessments/6zp6-pxei).

For detailed property assessment information, visit https://assessmentsearch.calgary.ca

In [2]:
Data = pd.read_csv('Calgary/Property_Assessments.csv')

#### Table of contents

* [Preprocessing](#Preprocessing)
    * [Assessed Value Column](#Assessed-Value-Column)
    * [Community Code and Community Name Columns](#Community-Code-and-Community-Name-Columns)
    * [Latitude, Longitude and Location Columns](#Latitude,-Longitude-and-Location-Columns)
    * [Changing the Dataframe Column Names to Title Case](#Changing-the-Dataframe-Column-Names-to-Title-Case)
    * [Adding More Details](#Adding-More-Details)
        * [Loading a geojson file](#Loading-a-geojson-file)
        * [Processing the geojson file](#Processing-the-geojson-file)
* [Processed Data](#Processed-Data)

# Preprocessing

The list of columns:

In [3]:
Columns_list = Data.columns.tolist()
print(Columns_list)

['ROLL_YEAR', 'ROLL_NUMBER', 'ADDRESS', 'ASSESSED_VALUE', 'ASSESSMENT_CLASS', 'ASSESSMENT_CLASS_DESCRIPTION', 'RE_ASSESSED_VALUE', 'NR_ASSESSED_VALUE', 'FL_ASSESSED_VALUE', 'COMM_CODE', 'COMM_NAME', 'LATITUDE', 'LONGITUDE', 'location', 'UNIQUE_KEY']


In [4]:
pd.DataFrame(Data.isnull().sum(),columns=['Number of NaN Values'])

,Number of NaN Values
ROLL_YEAR,0
ROLL_NUMBER,0
ADDRESS,186
ASSESSED_VALUE,1855
ASSESSMENT_CLASS,0
ASSESSMENT_CLASS_DESCRIPTION,0
RE_ASSESSED_VALUE,320420
NR_ASSESSED_VALUE,6448017
FL_ASSESSED_VALUE,6756853
COMM_CODE,2


As can be see, there are a consider number of **NaN** values that we need to deal with them before starting this study.

## Assessed Value Column

In [5]:
def Search_Columns(Inp):
    ''' Search for specific value (Inp) in the column list '''
    mylist=list()
    for i in range(len(Columns_list)):
        if Columns_list[i].find(Inp) != -1:
            mylist.append(Columns_list[i])
    return mylist

First off, let's create a DataFrame from rows that **Assessed Value** is **NaN**.

In [6]:
mylist = Search_Columns('VALUE')
Null = Data[Data.ASSESSED_VALUE.isna()][mylist]
Null.head()

,ASSESSED_VALUE,RE_ASSESSED_VALUE,NR_ASSESSED_VALUE,FL_ASSESSED_VALUE
517671,NaN,NaN,NaN,NaN
528056,NaN,NaN,NaN,NaN
528060,NaN,NaN,NaN,NaN
528061,NaN,NaN,NaN,NaN
528062,NaN,NaN,NaN,NaN


We would like to check whether there is a row that one of the last three columns has a non-NaN value.

In [7]:
Null[Null.iloc[:, 1:].notnull().all(axis=1)]

,ASSESSED_VALUE,RE_ASSESSED_VALUE,NR_ASSESSED_VALUE,FL_ASSESSED_VALUE


There is no such row, and we drop all of these rows.

In [8]:
Data.dropna(subset=['ASSESSED_VALUE'], how='all', inplace=True)

## Community Code and Community Name Columns

First off,

In [9]:
mylist = Search_Columns('COMM')
mylist1 = np.where(Data['COMM_NAME'].isna())[0].tolist()
Data.iloc[mylist1,:][mylist]

,COMM_CODE,COMM_NAME
1537788,12E,NaN
1538036,12E,NaN
1538125,12E,NaN
1538263,12E,NaN
1538386,12E,NaN
...,...,...
5785191,06B,NaN
6113118,14U,NaN
6114267,14U,NaN
6121987,06B,NaN


We can use the information on Community Name Column and fill Community Code column.

In [10]:
Temp = Data.copy()
Temp = Temp[Temp.COMM_CODE.notna()]
Temp = Temp.drop_duplicates('COMM_CODE')
Temp = Temp[mylist].reset_index(drop=True)
Temp.head()

,COMM_CODE,COMM_NAME
0,CIA,CALGARY INTERNATIONAL AIRPORT
1,SRI,SADDLE RIDGE INDUSTRIAL
2,SAD,SADDLE RIDGE
3,WSP,WEST SPRINGS
4,MRT,MARTINDALE


Now, we can this for completing **COMM_NAME** column

In [11]:
Counter = 0
Progress_Bar = progressbar.ProgressBar(maxval=len(mylist1), widgets=[progressbar.Bar('=', '|', '|'),
                                                                    progressbar.Percentage()])
Progress_Bar.start()
for i in mylist1:
    Counter+=1
    Progress_Bar.update(Counter)
    for j in range(len(Temp)):
        if Data.iloc[i,9]==Temp.iloc[j,0]:
            Data.iloc[i,10]=Temp.iloc[j,1]

|=========================================================================|100%

Now

In [12]:
Data.iloc[mylist1,:][mylist].head()

,COMM_CODE,COMM_NAME
1537788,12E,12E
1538036,12E,12E
1538125,12E,12E
1538263,12E,12E
1538386,12E,12E


However,

In [13]:
mylist = Search_Columns('COMM')
mylist1 = np.where(Data['COMM_NAME'].isna())[0].tolist()
Temp = Data.iloc[mylist1,:]['LATITUDE'].tolist()
Data.iloc[mylist1,:][mylist]

,COMM_CODE,COMM_NAME
4327764,NaN,NaN
4724088,NaN,NaN


and we label these rows as Others

In [14]:
Data.iloc[mylist1,9]='Other'
Data.iloc[mylist1,10]='Other'
Data.iloc[mylist1,:][mylist]

,COMM_CODE,COMM_NAME
4327764,Other,Other
4724088,Other,Other


Moreover, modifying **COMM_NAME** column entries.

In [15]:
Data['COMM_NAME'] = Data['COMM_NAME'].apply(lambda x: x.title())

## Latitude, Longitude and Location Columns 

For these columns, note that

In [16]:
mylist = np.where(Data['location'].isna())[0].tolist()
Null = Data[Data.location.isna()][['ROLL_NUMBER','ADDRESS','LATITUDE','LONGITUDE','location']]
print('The number of missing values: %i' % len(Null))
Null.head()

The number of missing values: 22277


,ROLL_NUMBER,ADDRESS,LATITUDE,LONGITUDE,location
131,202016192,47V 142 89 ST SW,NaN,NaN,NaN
856,202500559,171V 214 SHERWOOD SQ NW,NaN,NaN,NaN
889,202506515,65 30 AV SW,NaN,NaN,NaN
1693,202540308,417 20 WALGROVE WK SE,NaN,NaN,NaN
2250,202571543,2204 310 12 AV SW,NaN,NaN,NaN


We can use imputing with the mean strategy to recover these missing data in each community. Therefore,

In [17]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
mylist = Data.COMM_CODE.unique().tolist()
# Progressbar
Counter = 0
Progress_Bar = progressbar.ProgressBar(maxval=len(mylist),
                                       widgets=[progressbar.Bar('#', '|', '|'), progressbar.Percentage()])
Progress_Bar.start()

for i in mylist:
    # updating the progress bar
    Counter+=1
    Progress_Bar.update(Counter)
    # searching for the index of values in the community code column
    indx = np.where(Data['COMM_CODE']==i)[0].tolist()
    # LATITUDE
    Temp = Data.iloc[indx,:].LATITUDE
    if Temp.isnull().sum() < len(Temp):
        Temp = imp.fit_transform(Temp.values.reshape(-1, 1))
        Data.iloc[indx,11] = Temp
    del Temp
    # LONGITUDE
    Temp = Data.iloc[indx,:].LONGITUDE
    if Temp.isnull().sum() < len(Temp):
        Temp = imp.fit_transform(Temp.values.reshape(-1, 1))
        Data.iloc[indx,12] = Temp
    del Temp

del mylist, imp, Null

|######################################################################## | 98%

## Changing the Dataframe Column Names to Title Case

In [18]:
mylist = Data.columns.tolist()
mylist = [x.title() for x in mylist]
Data.columns = mylist
del mylist

## Adding More Details

### Loading a geojson file

In [19]:
Community_Boundaries = gpd.read_file('Calgary/Community_Boundaries.geojson')
# Chaning the headers to title
mylist = Community_Boundaries.columns.tolist()
mylist = [x.title() for x in mylist]
mylist = [x.replace('Geometry','geometry') for x in mylist]
Community_Boundaries.columns = mylist
del mylist

In [20]:
Community_Boundaries.head(4)

,Comm_Structure,Name,Sector,Class_Code,Srg,Class,Comm_Code,geometry
0,INNER CITY,SUNALTA,CENTRE,1,BUILT-OUT,Residential,SNA,"POLYGON ((-114.10061493005 51.047994301993, -1..."
1,BUILDING OUT,WEST SPRINGS,WEST,1,DEVELOPING,Residential,WSP,"POLYGON ((-114.187805990867 51.061506344418, -..."
2,OTHER,12A,SOUTHEAST,4,N/A,Residual Sub Area,12A,"POLYGON ((-113.869446537594 50.979572332142, -..."
3,1950s,WINDSOR PARK,CENTRE,1,BUILT-OUT,Residential,WND,"POLYGON ((-114.081517547811 51.008716296692, -..."


### Processing the geojson file

In [21]:
Temp = Community_Boundaries.drop(columns=['Name','geometry'])
N = len(Temp.iloc[:,:-1].columns.tolist())
Counter = 0
Progress_Bar = progressbar.ProgressBar(maxval=N, widgets=[progressbar.Bar('=', '|', '|'), progressbar.Percentage()])
Progress_Bar.start()
for i in range(N):
    Counter+=1
    Progress_Bar.update(Counter)
    Temp.iloc[:,i] = Temp.iloc[:,i].str.title()
Progress_Bar.finish()

|=========================================================================|100%


In [22]:
mylist = Data.Comm_Code.unique().tolist()
Temp = Temp[Temp.Comm_Code.isin(mylist)]
Temp1 = pd.merge(Data, Temp, how ='inner', on ='Comm_Code')
Data = copy.deepcopy(Temp1)  
del Temp, Temp1

# Processed Data

In [23]:
Data.head()

,Roll_Year,Roll_Number,Address,Assessed_Value,Assessment_Class,Assessment_Class_Description,Re_Assessed_Value,Nr_Assessed_Value,Fl_Assessed_Value,Comm_Code,Comm_Name,Latitude,Longitude,Location,Unique_Key,Comm_Structure,Sector,Class_Code,Srg,Class
0,2019,2004505,3420 80 AV NE,963500.0,NR,Non-residential,NaN,963500.0,NaN,CIA,Calgary International Airport,51.125707,-113.985934,"(51.1257068226041, -113.985933504339)",2019002004505,Other,Northeast,2,N/A,Industrial
1,2019,200613560,8063 22 ST NE,3160000.0,NR,Non-residential,NaN,3160000.0,NaN,CIA,Calgary International Airport,51.123612,-114.011797,"(51.1236116266249, -114.011796685429)",2019200613560,Other,Northeast,2,N/A,Industrial
2,2019,200613578,24 AERO DR NE,11690000.0,NR,Non-residential,NaN,11690000.0,NaN,CIA,Calgary International Airport,51.123612,-114.011797,"(51.1236116266249, -114.011796685429)",2019200613578,Other,Northeast,2,N/A,Industrial
3,2019,200693448,37 AERO DR NE,11450000.0,NR,Non-residential,NaN,11450000.0,NaN,CIA,Calgary International Airport,51.123612,-114.011797,"(51.1236116266249, -114.011796685429)",2019200693448,Other,Northeast,2,N/A,Industrial
4,2019,200929172,1902 MCCALL LD NE,3010000.0,NR,Non-residential,NaN,3010000.0,NaN,CIA,Calgary International Airport,51.123612,-114.011797,"(51.1236116266249, -114.011796685429)",2019200929172,Other,Northeast,2,N/A,Industrial


In [24]:
pd.DataFrame(Data.isnull().sum(),columns=['Number of NaN Values'])

,Number of NaN Values
Roll_Year,0
Roll_Number,0
Address,186
Assessed_Value,0
Assessment_Class,0
Assessment_Class_Description,0
Re_Assessed_Value,313753
Nr_Assessed_Value,6431949
Fl_Assessed_Value,6739847
Comm_Code,0


### Saving

In [25]:
Data.to_csv('Calgary/Property_Assessments_Clean.csv',index=False)

***